In [6]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

def extract_audio_features(audio_files_path):
    features = []
    for subdir, dirs, files in os.walk(audio_files_path):
        for file in files:
            if file.endswith('.mp3'):
                audio_path = os.path.join(subdir, file)
                feature_dict = process_audio_file(audio_path, file)
                if feature_dict:  # Ensure no failed extraction
                    features.append(feature_dict)
    return pd.DataFrame(features)

def process_audio_file(audio_path, file):
    try:
        print(f"Processing file: {audio_path}")
        audio_data, sr = librosa.load(audio_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=13)
        S = librosa.feature.melspectrogram(y=audio_data, sr=sr,n_mels=50)
        tempogram = librosa.feature.tempogram(onset_envelope=librosa.onset.onset_strength(y=audio_data, sr=sr), sr=sr)

        # Compile feature dictionary including all MFCCs, Mel spectrogram, and Tempogram values
        feature_dict = {'filename': file}
        feature_dict.update({f'mfcc_{i}': mfcc.mean(axis=1)[i] for i in range(mfcc.shape[0])})
        feature_dict.update({f'mel_{i}': S.mean(axis=1)[i] for i in range(S.shape[0])})
        feature_dict.update({f'tempo_{i}': tempogram.mean(axis=1)[i] for i in range(tempogram.shape[0])})

        return feature_dict
    except Exception as e:
        print(f"Error processing {file}: {e}")
        return None

def normalize_features(df):
    print("Normalizing features...")
    non_numeric = df[['filename']]
    numeric_df = df.select_dtypes(include=[np.number])
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(numeric_df)
    scaled_df = pd.DataFrame(scaled_features, columns=numeric_df.columns, index=df.index)
    return pd.concat([non_numeric, scaled_df], axis=1)

def classify_rhythmic_features(df, n_clusters=10):
    print("Classifying features based on rhythmic patterns...")
    rhythm_columns = [col for col in df.columns if col.startswith('tempo')]
    kmeans = KMeans(n_clusters=n_clusters)
    df['cluster'] = kmeans.fit_predict(df[rhythm_columns])
    return df

def stratified_sample(df, sample_size_per_group=200):
    print("Performing stratified sampling...")
    return df.groupby('cluster').apply(lambda x: x.sample(min(len(x), sample_size_per_group))).reset_index(drop=True)

def save_data_to_csv(df, file_path):
    df.to_csv(file_path, index=False)
    print(f"Data successfully saved to {file_path}")

if __name__ == "__main__":
    audio_files_path = '/home/mutayyab/Documents/dataset'
    features_df = extract_audio_features(audio_files_path)
    if not features_df.empty:
        normalized_features_df = normalize_features(features_df)
        classified_df = classify_rhythmic_features(normalized_features_df)
        sampled_df = stratified_sample(classified_df)
        save_data_to_csv(sampled_df, '/home/mutayyab/Documents/dataset/preprocessed_audios.csv')


Processing file: /home/mutayyab/Documents/dataset/093/093007.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093964.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093406.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093960.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093002.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093448.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093829.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093885.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093832.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093810.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093591.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093345.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093059.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093456.mp3
Processing file: /home/mutayyab/Documents/dataset/093/093422.mp3
Processing file: /home/mu

/home/mutayyab/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Performing stratified sampling...
Data successfully saved to /home/mutayyab/Documents/dataset/preprocessed_audios.csv
